In [44]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

In [45]:
datadir ='/Users/francesco/Desktop/Thesis/Data/'

# Read the dataframes
df_bed = xr.open_dataset(datadir + 'BedMachineAntarctica-v3.nc')
df_melt = xr.open_dataset(datadir + 'bmelt_bmachine_grid_v1.nc')

#Adding the names of the glaciers, in order to select later the interesting ones
df_names = pd.read_csv(datadir + 'Merged_Integrated_melt_rates.csv')

In [46]:
#Extracting the variables

thickness = df_bed.thickness #dim -> (x,y)
melt = df_melt.melt #dim -> (t,x,y)

In [47]:
x = df_bed.x
y = df_bed.y
time = df_melt.time

In [48]:
# Plot the bed with the meshgrid

''' 
#Creatinng the mesh grid
coord = x.values
X, Y = np.meshgrid(coord, coord)


plt.figure(figsize=(10,10))
plt.pcolormesh(X, Y, bed, cmap='gist_earth')
plt.colorbar()
plt.show()
'''


" \n#Creatinng the mesh grid\ncoord = x.values\nX, Y = np.meshgrid(coord, coord)\n\n\nplt.figure(figsize=(10,10))\nplt.pcolormesh(X, Y, bed, cmap='gist_earth')\nplt.colorbar()\nplt.show()\n"

In [49]:
#Now I want to select the areas of the region of interest, and there I want to calculate the basal melt
#just for the thickest part of the glacier, i.e. the part of the glacier where the thickness is above the mean value

In [50]:
import glob
import pandas as pd
import os
from shapely.geometry import shape
import csv
#from mapxy import mapxy
import pyproj
#from ps_latlon_conversion import xyscale_south
import pyproj
#import ogr
from osgeo import gdal, osr
import matplotlib.pyplot as plt
import fiona
import rasterio
import rasterio.transform
import rasterio.mask
from fiona import Feature, Geometry
from shapely.geometry import mapping, shape

In [51]:
#get information from the shp file

path_to_tif = '/Users/francesco/Desktop/Data/GEOTIFFs/'
shapefile_path = '/Users/francesco/Desktop/Thesis/Data/ice_shelf.shp'

ids = []
Names = []
lat_lon_coords = []
regions = []
boundaries = []
areas = []


#Here i just uses the first tif file to get the information, so 1992
file_tif = path_to_tif + 'melt_' + str(1992) + '_warp_ps.tif'
print(file_tif)


shapefile=fiona.open(shapefile_path)
print(shapefile.schema['properties'])


for feature in shapefile:


    id = feature['id']
    name=feature['properties']['name']
    geometry=feature['geometry']
    region = feature['properties']['regions']
    lat = feature['properties']['latitude']
    lon = feature['properties']['longitude']
    #thickness = feature['properties']['thickness_']

    bounds=shape(geometry).bounds
    xmin=bounds[0];ymin=bounds[1];xmax=bounds[2];ymax=bounds[3]

    area_temp = np.abs((xmax-xmin)*(ymax-ymin))

    #Taking the info
    ids.append(id)
    Names.append(name)
    lat_lon_coords.append([lat,lon])
    regions.append(region)
    boundaries.append([xmin,xmax,ymin,ymax])
    areas.append(area_temp)


/Users/francesco/Desktop/Data/GEOTIFFs/melt_1992_warp_ps.tif
{'id': 'int:10', 'name': 'str:30', 'regions': 'str:11', 'type': 'str:2', 'is_index': 'int:10', 'ice_shelf_': 'str:50', 'measures_n': 'str:50', 'latitude': 'float:24.15', 'longitude': 'float:24.15', 'thickness_': 'float:24.15', 'thicknes_1': 'float:24.15', 'smb_mean_9': 'float:24.15', 'smb_mean_1': 'float:24.15', 'ss_gl_flux': 'float:24.15', 'ss_gl_fl_1': 'float:24.15', 'ss_calving': 'float:24.15', 'ss_calvi_1': 'float:24.15', 'model_gl_f': 'float:24.15', 'instant_co': 'float:24.15', 'mass_loss_': 'float:24.15', 'mass_los_1': 'float:24.15', 'mass_los_2': 'float:24.15', 'mass_los_3': 'float:24.15', 'mass_los_4': 'float:24.15', 'mass_los_5': 'float:24.15', 'mass_los_6': 'float:24.15', 'mass_los_7': 'float:24.15', 'mass_los_8': 'float:24.15', 'mass_los_9': 'float:24.15', 'mass_los10': 'float:24.15', 'mass_los11': 'float:24.15', 'mass_los12': 'float:24.15', 'mass_los13': 'float:24.15', 'mass_los14': 'float:24.15', 'mass_los15': 'f

In [52]:
#This block of code is to calculate the mean thickness and the mean melt for the thickest part of the glacier.
#Is commented since the output is already saved in the csv file


#The idea is to create a loop that goes through all the glaciers, 18 minuti
#and for each glacier it creates a mask, and then it calculates the mean thickness and the mean melt for the thickest part of the glacier

''' 
Years = np.arange(1992, 2018, 1)
thickness_mean = np.zeros(len(ids))
melt_mean = np.zeros((len(ids), len(time)))
print(np.shape(melt_mean))


for glacier in range(0, len(ids)):
    x_min , x_max, y_min, y_max =  boundaries[glacier]
    mask = (x > x_min) & (x < x_max) & (y > y_min) & (y < y_max)
    print('Glacier number: ', glacier)
    
    # index both thickness and melt using the mask
    thickness_roi = thickness.where(mask, drop=True)
    thickness_mean_roi = np.mean(thickness_roi)
    thickness_mean[glacier] = thickness_mean_roi

    
    for t in range(0, len(time)):
        melt_roi = melt[t].where(mask, drop=True)
        melt_mean_temp = np.nanmean(melt_roi.where(thickness_roi > thickness_mean_roi))
        melt_mean[glacier, t] = melt_mean_temp

'''



" \nYears = np.arange(1992, 2018, 1)\nthickness_mean = np.zeros(len(ids))\nmelt_mean = np.zeros((len(ids), len(time)))\nprint(np.shape(melt_mean))\n\n\nfor glacier in range(0, len(ids)):\n    x_min , x_max, y_min, y_max =  boundaries[glacier]\n    mask = (x > x_min) & (x < x_max) & (y > y_min) & (y < y_max)\n    print('Glacier number: ', glacier)\n    \n    # index both thickness and melt using the mask\n    thickness_roi = thickness.where(mask, drop=True)\n    thickness_mean_roi = np.mean(thickness_roi)\n    thickness_mean[glacier] = thickness_mean_roi\n\n    \n    for t in range(0, len(time)):\n        melt_roi = melt[t].where(mask, drop=True)\n        melt_mean_temp = np.nanmean(melt_roi.where(thickness_roi > thickness_mean_roi))\n        melt_mean[glacier, t] = melt_mean_temp\n\n"

In [53]:
# save as CSV file

''' 
Years = np.arange(1992, 2018, 1)
np.savetxt('thickness_and_basal_mean.csv', np.column_stack((thickness_mean, melt_mean)), delimiter=',')
np.savetxt('melt_mean_1.csv', melt_mean, delimiter=',', header=','.join(map(str, Years)))
np.savetxt('thickness_mean_1.csv', thickness_mean, delimiter=',', header=','.join(map(str, Names)))

'''

" \nYears = np.arange(1992, 2018, 1)\nnp.savetxt('thickness_and_basal_mean.csv', np.column_stack((thickness_mean, melt_mean)), delimiter=',')\nnp.savetxt('melt_mean_1.csv', melt_mean, delimiter=',', header=','.join(map(str, Years)))\nnp.savetxt('thickness_mean_1.csv', thickness_mean, delimiter=',', header=','.join(map(str, Names)))\n\n"

In [54]:
#reading the csv file and extracting the variables
data_dir_2 = '/Users/francesco/Desktop/Thesis/Bed_machine/'

melt_mean = pd.read_csv(data_dir_2 + 'melt_mean_1.csv')
thickness_mean = pd.read_csv(data_dir_2 + 'thickness_mean_1.csv')



#Here I import the dataset for the glaciers that I am interested in

df_names = pd.read_csv(datadir + 'Merged_Integrated_melt_rates.csv')
Glaciers = df_names['Names']

interesting_glaciers = ['Pine_Island', 'Thwaites', 'Crosson', 'Dotson', 'Getz_2', 'Venable', 'Getz', 'Getz_1' ]
interesting_indices = np.where(np.isin(Glaciers, interesting_glaciers))[0]


In [55]:
print(interesting_indices)
print(Glaciers[interesting_indices])
len(Glaciers[interesting_indices])

[43 44 52 61 67 68 69 71]
43         Getz_2
44           Getz
52         Getz_1
61        Venable
67    Pine_Island
68       Thwaites
69        Crosson
71         Dotson
Name: Names, dtype: object


8

In [56]:
#Take the first raw of the dataframe melt_mean


df_interesting_glaciers = melt_mean.iloc[interesting_indices]
df_interesting_glaciers


,# 1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,-8.413609,-5.942461,-6.794760,-4.981510,-6.548600,-6.890209,-7.231894,-4.604581,-7.584861,-4.915061,...,-7.526445,-4.309380,-4.538844,-5.970263,-7.362037,-7.732249,-6.972290,-5.390824,-5.946024,-3.685726
52,-1.823234,-1.201954,-3.142510,-2.670408,-6.359390,-5.159474,-2.637508,-4.361383,-1.923420,-4.688521,...,1.050775,-3.614895,-4.335759,-6.819175,-3.618371,-6.221341,-6.418642,-4.989903,-1.323176,-7.514523
61,-3.605442,-2.884086,-2.468552,-4.068516,-4.463484,-1.865358,-6.969004,-5.466586,-4.411549,-4.049332,...,-6.059647,-4.980170,-4.262447,-0.610405,-3.866952,-2.053005,-2.829292,-1.272592,-4.511341,-1.349952
67,-21.449914,-20.570059,-19.899727,-23.212189,-20.787389,-19.075990,-22.901829,-18.525763,-20.380371,-21.462235,...,-24.523703,-19.307042,-20.636665,-17.018780,-13.666499,-17.654303,-16.968275,-15.354581,-16.816891,-17.005557
68,-24.447653,-32.818858,-28.520485,-36.063666,-29.334189,-31.533917,-36.432817,-24.824092,-29.109719,-30.906729,...,-42.560500,-32.235597,-29.586395,-22.604142,-2.746458,-2.770944,-4.112532,-1.635843,-1.486520,-2.152010
69,-15.067985,-21.842386,-16.456872,-23.358443,-19.444568,-16.617810,-21.323802,-18.283064,-19.254370,-20.444287,...,-22.635928,-13.707167,-15.193405,-11.320781,-11.235661,-10.193431,-14.754618,-8.731806,-8.022121,-9.182577
71,-13.143792,-17.026532,-13.825222,-18.209323,-15.392142,-9.383227,-15.989702,-13.187187,-16.124847,-15.330571,...,-19.842242,-14.658020,-12.541165,-11.534834,-11.247263,-10.096984,-12.121224,-9.265471,-8.149614,-7.859727


In [57]:
#Calculate a linear regression of the melt rate for each glacier

from scipy import stats

Years = np.arange(1992, 2018, 1)

lin_coeff = np.zeros((len(df_interesting_glaciers),2))
lin_reg = np.zeros((len(df_interesting_glaciers),len(Years)))
r_2 = np.zeros(len(df_interesting_glaciers))

for glacier in range(0, len(df_interesting_glaciers)):
    lin_coeff[glacier,:] = np.polyfit(Years, df_interesting_glaciers.iloc[glacier], 1)
    lin_reg[glacier,:] = np.polyval(lin_coeff[glacier,:], Years)
    r_2[glacier] = stats.linregress(Years, df_interesting_glaciers.iloc[glacier])[2]**2


In [66]:
#Create a pdf file with the plots, one per page

from matplotlib.backends.backend_pdf import PdfPages

pp = PdfPages('Melt_rates.pdf')

for glacier in range(0, len(df_interesting_glaciers)):
    fig = plt.figure()
    plt.plot(Years, -df_interesting_glaciers.iloc[glacier], label='Melt rate')
    plt.plot(Years, -lin_reg[glacier,:], label='Linear regression')
    plt.title(Glaciers[interesting_indices[glacier]])
    plt.xlabel('Years')
    plt.ylabel('Melt rate [m/yr]')
    plt.legend()
    pp.savefig(fig)
    plt.close()

pp.close()